In [26]:
## 0. Import libraries
import os
import shutil
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [27]:
# ## 1. Move all files in the PNEUMONIA folder to the respective BACTERIA and VIRUS folders
# # Paths
# base_dir = "../dataset/multiclass"
# splits = ['train', 'val', 'test']
# categories = ["bacteria", "virus"]

# for split in splits:
#     pneumonia_path = os.path.join(base_dir, split, "PNEUMONIA")
    
#     # list all files in the pneumonia folder
#     files = os.listdir(pneumonia_path)
#     os.makedirs(os.path.join(base_dir, split, "BACTERIA"), exist_ok=True)
#     os.makedirs(os.path.join(base_dir, split, "VIRUS"), exist_ok=True)
    
#     for file in files:
#         if "_bacteria_" in file:
#             shutil.move(os.path.join(pneumonia_path, file), os.path.join(base_dir, split, "BACTERIA", file))
#         elif "_virus_" in file:
#             shutil.move(os.path.join(pneumonia_path, file), os.path.join(base_dir, split, "VIRUS", file))
            
#     # Remove the now-empty PNEUMONIA folder
#     shutil.rmtree(pneumonia_path)


In [28]:
## 2. Load the datasets and check the class distribution
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Load datasets
# train_data = datasets.ImageFolder(root='/kaggle/input/chest-xray-pneumonia/chest_xray/train', transform=transform)
# val_data = datasets.ImageFolder(root='/kaggle/input/chest-xray-pneumonia/chest_xray/val', transform=transform)
# test_data = datasets.ImageFolder(root='/kaggle/input/chest-xray-pneumonia/chest_xray/test', transform=transform)
train_data = datasets.ImageFolder(root='../dataset/multiclass/train', transform=transform)
# val_data = datasets.ImageFolder(root='../dataset/multiclass/val', transform=transform)
test_data = datasets.ImageFolder(root='../dataset/multiclass/test', transform=transform)

# Data loaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
# val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Check whether the classes are balanced
print("Train data: total samples -", len(train_data))
for i in range(len(train_data.class_to_idx)):
    print(f"Class {i} - {train_data.classes[i]}: {train_data.targets.count(i)}")
print("\nTest data: total samples -", len(test_data))
for i in range(len(test_data.class_to_idx)):
    print(f"Class {i} - {test_data.classes[i]}: {test_data.targets.count(i)}")


Train data: total samples - 5216
Class 0 - BACTERIA: 2530
Class 1 - NORMAL: 1341
Class 2 - VIRUS: 1345

Test data: total samples - 624
Class 0 - BACTERIA: 242
Class 1 - NORMAL: 234
Class 2 - VIRUS: 148


In [29]:
## 3. Define the CNN model
class PneumoniaCNN(nn.Module):
    def __init__(self):
        super(PneumoniaCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout(0.25)
        
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.dropout2 = nn.Dropout(0.25)
        
        self.conv5 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.dropout3 = nn.Dropout(0.4)
        
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.dropout4 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 3)  # Three classes: NORMAL, BACTERIA, VIRUS

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = self.dropout1(x)
        
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        x = self.dropout2(x)
        
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = self.pool(x)
        x = self.dropout3(x)
        
        x = x.view(-1, 128 * 28 * 28)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.dropout4(x)
        x = self.fc2(x)
        return x


In [30]:
## 4. Train the model
if torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PneumoniaCNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step( )
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")


Device: mps
Epoch 1, Loss: 0.8416714421444875
Epoch 2, Loss: 0.5832753834183231
Epoch 3, Loss: 0.5211738724650049
Epoch 4, Loss: 0.48080698185903165
Epoch 5, Loss: 0.4647474778027622
Epoch 6, Loss: 0.43704420437842056
Epoch 7, Loss: 0.423778795940013
Epoch 8, Loss: 0.4054294563144263
Epoch 9, Loss: 0.3859456843393712
Epoch 10, Loss: 0.38167435579870373


In [31]:
## 5. Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total}%")


Test Accuracy: 69.23076923076923%


In [32]:
## 6. Save the model
# Save the model's state_dict
# torch.save(model.state_dict(), "pneumonia_cnn.pth")
torch.save(model, "pneumonia_cnn.pth")


In [33]:
## 7. Load the model and evaluate it
model = torch.load("pneumonia_cnn.pth").to(device)
model.eval()

# Evaluate the model
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total}%")


Test Accuracy: 69.23076923076923%
